In [40]:
# import
import torch
import torch.nn as nn
from itertools import repeat

import numpy as np
import pandas as pd
from Utils import custom_data_loader, preprocess_data
from Utils.SummaryWriter import LogSummary as LS
from Models.simpleFFBNN import SimpleFFBNN



In [16]:
def get_device():
    """Function to get the device to be used for training the model
    """
    cuda = torch.cuda.is_available()
    print("CUDA Available: ", cuda)

    if cuda:
        gpu = GPUtil.getFirstAvailable()
        print("GPU Available: ", gpu)
        torch.cuda.set_device(gpu)
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    print("Device: ", device)
    return device

device = get_device()


CUDA Available:  False
Device:  cpu


In [2]:
# load data
dataloader_train, dataloader_test, dataloader_val = preprocess_data(pd.read_csv('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Data/quality_of_food.csv'))


/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Utils/config.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.X['savings'] = np.where(self.X['savings'] == 'low', 0, np.where(self.X['savings'] == 'medium', 1, 2))


In [3]:
model = SimpleFFBNN(4, 1)

In [ ]:
checkpoint = torch.load('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/trainedModels/simple_model.pth', map_location=torch.device('cpu'))
#print(checkpoint)
#model.load_state_dict(checkpoint['model'])
model.load_state_dict(checkpoint)
    

In [54]:
class Saveoutput():
    def __init__(self, instances, batch_size, round):
        self.T = instances
        self.batch_size = batch_size
        self.outputs = []
        self.round = round
        self.counter = 0


    def __call__(self, module, module_in, module_out):
        if self.counter < 3:
            sample_data = np.random.randint(self.batch_size)
            outs = module_out.view(self.T, self.batch_size, -1)[:, 0, :]
            layer_size = outs.shape[1]

            
            LS.per_round_layer_output(layer_size, outs, self.round)

            self.counter += 1


    def clear(self):
        self.outputs = []
        


        

In [53]:
def trainSeedModelClosedForm(model, dataloader_train, dataloader_test, dataloader_val, num_epochs, learning_rate, batch_size, round):
    
    # loss function
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # load the trained model
    model = SimpleFFBNN(4, 1)
    checkpoint = torch.load('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/trainedModels/simple_model.pth', map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint)

    hook_handles = []
    save_output = Saveoutput(1, batch_size, round)
    model.eval()
    for layer in model.kl_layers:
        hook_handles.append(layer.register_forward_hook(save_output))

    with torch.no_grad():
        for batch_index, (X, y) in enumerate(dataloader_train):
            batch_size = X.shape[0]
            save_output.batch_size = batch_size
            X = X.repeat(1, 1)
            y = y.repeat(1,1)
            X, y = X.to(device), y.to(device)
            y_pred = model(X)

        save_output.clear()
        save_output.counter = 0
        for handle in hook_handles:
            handle.remove()

    #write_summary = LogSummary()




# use the function
output =trainSeedModelClosedForm(model, dataloader_train, dataloader_test, dataloader_val, 100, 0.0001, 32)
    

TypeError: per_round_layer_output() missing 1 required positional argument: 'round'

In [22]:
print(output)

None


In [8]:
def trainSeedModelClosedForm(self):
    hook_handles = []
    save_output = saveOutput(self.instances, self.batch_size, 0)
    self.model.eval()
    for layer in self.model.layers:
        hook_handles.append(layer.register_forward_hook(save_output))
    
    with torch.no_grad():
        for batch_index, (X, y) in enumerate(self.data_train):
            batch_size = X.shape[0]
            save_output.batch_size = batch_size
            X = X.repeat(self.instances, 1)
            y = y.repeat(self.instances)
            X, y = X.to(self.device), y.to(self.device)
            ensemble_outputs = self.model(X)

        save_output.clear()
        save_output.counter = 0
        for handle in hook_handles:
            handle.remove()
